In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('bus_demand.csv')

In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.groupby(['municipality_id', pd.Grouper(key='timestamp', freq='1H')]).max().reset_index()

In [7]:
df = df.set_index(['municipality_id', 'timestamp']).groupby('municipality_id').apply(lambda x: x.interpolate(method='linear', limit_direction='both'))
df = df.reset_index()


C:\Users\mx\AppData\Local\Temp\ipykernel_1780\4020780954.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.set_index(['municipality_id', 'timestamp']).groupby('municipality_id').apply(lambda x: x.interpolate(method='linear', limit_direction='both'))


In [8]:
train_start = '2017-06-24'
train_end = '2017-08-04'
test_start = '2017-08-05'
test_end = '2017-08-19'

train_mask = (df['timestamp'] >= train_start) & (df['timestamp'] <= train_end)
test_mask = (df['timestamp'] >= test_start) & (df['timestamp'] <= test_end)

train = df.loc[train_mask]
test = df.loc[test_mask]


In [9]:
from sklearn.metrics import mean_absolute_error

def evaluate(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)


In [2]:
def baseline_model(df_train, df_test):
    # Get a list of all municipalities in the training set
    municipalities = df_train['municipality_id'].unique()

    predictions = []
    for i, row in df_test.iterrows():
        if i == 0:
            predictions.append(row['usage'])
        else:
            prev_row = df_test.iloc[i-1]
            if row['municipality_id'] not in municipalities:
                # Skip prediction for this municipality
                predictions.append(row['usage'])
            elif row['municipality_id'] == prev_row['municipality_id']:
                predictions.append(prev_row['usage'])
            else:
                predictions.append(row['usage'])
    return np.array(predictions)


In [15]:
#!pip install pmdarima

from pmdarima.arima import auto_arima

def sarima_model(df):
    predictions = []
    for municipality_id, group in df.groupby('municipality_id'):
        train_group = group.loc[train_mask]
        test_group = group.loc[test_mask]

        # Use pmdarima to select the best hyperparameters
        model = auto_arima(train_group['usage'], seasonal=True, m=24, suppress_warnings=True)
        model.fit(train_group['usage'])

        # Forecast the test set
        forecast = model.predict(len(test_group))
        predictions.extend(forecast)

    return np.array(predictions)

# Evaluate the SARIMA model on the test set
y_true = test['municipality_id']
y_pred = sarima_model(df)
error = evaluate(y_true, y_pred)
print(f'SARIMA model error: {error:.2f}')


c:\Users\mx\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\mx\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
